In [7]:
import dynamiqs as dq
import jax.numpy as jnp
import numpy as np
from sklearn.linear_model import LinearRegression

In [9]:
class state_sampler:
    def __init__(self, state, noise=False):
        self.dim = state.dims[0]
        self.state = state
        self.X, self.Y, self.W = dq.wigner(state)
        self.noise = noise

    def get_sample(self,x):
        ndim = len(np.array(x).shape)
        if ndim==1:
            val = np.float32(sampler.W[np.argmin(np.abs(self.X-x[0]))][np.argmin(np.abs(self.Y-x[1]))])
            return val + self.noise * np.random.rand() if self.noise else val
        elif ndim==2:
            return np.array([self.get_sample(y) for y in x])
        print(f"NOPE ndim={ndim}")
        return
        

In [29]:
def ketbra(n,m,dim=5):
    if n == m:
        return dq.fock(dim,n)@dq.fock(dim,m).dag()
    elif n < m:
        return (dq.fock(dim,n)@dq.fock(dim,m).dag() + dq.fock(dim,m)@dq.fock(dim,n).dag())/np.sqrt(2)
    else: # n > m
        return -1j*(dq.fock(dim,n)@dq.fock(dim,m).dag() - dq.fock(dim,m)@dq.fock(dim,n).dag())/np.sqrt(2)

def get_dm(rho):
    dim = round(np.sqrt(len(rho)))
    return dq.asqarray(np.sum([rho[n+m*dim] * ketbra(n,m,dim) for m in range(dim) for n in range(dim)], axis=0))

def get_vec(rho_dm):
    dim = len(rho_dm)
    return [np.trace(ketbra(n,m,dim)@rho_dm) for m in range(dim) for n in range(dim)]

class Kerneliser:
    def __init__(self, dim=5):
        self.dim = dim
        self.X, self.Y, _ = dq.wigner(ketbra(0,0, dim=dim))
        self.wigners = [[dq.wigner(ketbra(n,m,dim=dim))[2] for m in range(dim)] for n in range(dim)]

    def get_kernel(self, x):
        ndim = len(np.array(x).shape)
        if ndim==1:
            idxs = [np.argmin(np.abs(self.X-x[0])), np.argmin(np.abs(self.Y-x[1]))]
            return np.array([self.wigners[n][m][idxs[0]][idxs[1]] for m in range(self.dim) for n in range(self.dim)])
        elif ndim==2:
            return np.array([self.get_kernel(y) for y in x])
        print(f"NOPE ndim={ndim}")
        return

In [30]:
dim = 7
kerneliser = Kerneliser(dim)

In [31]:
n_points = 250
sampler = state_sampler(dq.coherent(dim,1), 0.5)

x_training = np.random.uniform(-2, 2, n_points * 2).reshape(-1, 2)
y_training = sampler.get_sample(x_training)
phi_training = kerneliser.get_kernel(x_training)

In [32]:
reg = LinearRegression().fit(phi_training, y_training)
dm_out = get_dm(reg.coef_)
dm_out /= dq.norm(dm_out)
dq.norm(dm_out)

Array(1.0000001, dtype=float32)

In [33]:
dq.fidelity(sampler.state, dm_out)

Array(0.7059199, dtype=float32)